In [0]:
from keras import backend
from keras import datasets
import keras
import numpy as np
from keras import models, layers
from keras.models import Model,Sequential, model_from_json
from keras.layers import Dense, Conv2D, AveragePooling2D, Flatten
from keras.datasets import mnist
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
# Load dataset as train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

img_rows, img_cols = x_train.shape[1:]

if backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')

# Set numeric type to float32 from uint8
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize value to [0, 1]
x_train /= 255
x_test /= 255

# Transform lables to one-hot encoding
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# Reshape the dataset into 4D array
x_train = x_train.reshape(x_train.shape[0], 32,32,1)
x_test = x_test.reshape(x_test.shape[0], 32,32,1) 

11493376/11490434 [==============================] - 1s 0us/step


In [0]:
#Instantiate an empty model
model = Sequential()

# C1 Convolutional Layer
model.add(layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(32,32,1)))

# S2 Pooling Layer
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

# C3 Convolutional Layer
model.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid'))

# S4 Pooling Layer
model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

#Flatten the CNN output so that we can connect it with fully connected layers
model.add(layers.Flatten())

# C5 Fully Connected Convolutional Layer
model.add(layers.Dense(120, activation='tanh'))

# FC6 Fully Connected Layer
model.add(layers.Dense(84, activation='tanh'))

#Output Layer with softmax activation
model.add(layers.Dense(10, activation='softmax'))

# Compile the model
model.compile(loss=keras.losses.categorical_crossentropy, optimizer='Adam', metrics=["accuracy"]) 
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 6)         156       
_________________________________________________________________
average_pooling2d_1 (Average (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 16)        2416      
_________________________________________________________________
average_pooling2d_2 (Average (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               48120     
_________________________________________________________________
dens

In [0]:
hist = model.fit(x=x_train,y=y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test), verbose=1) 
# Write the testing input and output variables
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 45s 756us/step - loss: 0.3301 - acc: 0.9030 - val_loss: 0.1496 - val_acc: 0.9541
Epoch 2/10
60000/60000 [==============================] - 45s 747us/step - loss: 0.1168 - acc: 0.9646 - val_loss: 0.0907 - val_acc: 0.9697
Epoch 3/10
60000/60000 [==============================] - 45s 753us/step - loss: 0.0776 - acc: 0.9764 - val_loss: 0.0697 - val_acc: 0.9784
Epoch 4/10
60000/60000 [==============================] - 45s 753us/step - loss: 0.0578 - acc: 0.9821 - val_loss: 0.0619 - val_acc: 0.9800
Epoch 5/10
60000/60000 [==============================] - 45s 744us/step - loss: 0.0465 - acc: 0.9853 - val_loss: 0.0578 - val_acc: 0.9817
Epoch 6/10
60000/60000 [==============================] - 45s 752us/step - loss: 0.0388 - acc: 0.9876 - val_loss: 0.0551 - val_acc: 0.9815
Epoch 7/10
60000/60000 [==============================] - 45s 756us/

In [0]:
# serialize model to JSON
model_json = model.to_json()

# Write the file name of the model

with open("Lenet.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
# Write the file name of the weights

model.save_weights("Lenet.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
json_file = open('Lenet.json', 'r')
lenet_model_json = json_file.read()
json_file.close()
lenet_model = model_from_json(lenet_model_json)
lenet_model.load_weights("Lenet.h5")
lenet_model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])
print("lenet model from disk")

lenet model from disk


In [0]:
# Write the testing input and output variables
score = lenet_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.05082243042171467
Test accuracy: 0.9843


In [0]:
lenet_model.load_weights("Lenet.h5")
lenet_model.compile(loss='categorical_crossentropy',optimizer='SGD',metrics=['accuracy'])
weights = lenet_model.layers[6].get_weights()
for i in range(0,8):
    if len(lenet_model.layers[i].get_weights())!=0:
        print("Layer Number:",i,"Layer Shape: ",lenet_model.layers[i].get_weights()[0].shape)

Layer Number: 0 Layer Shape:  (5, 5, 1, 6)
Layer Number: 2 Layer Shape:  (5, 5, 6, 16)
Layer Number: 5 Layer Shape:  (400, 120)
Layer Number: 6 Layer Shape:  (120, 84)
Layer Number: 7 Layer Shape:  (84, 10)


In [0]:
print(lenet_model.layers)
for i in range(len(lenet_model.layers)):
 if len(lenet_model.layers[i].get_weights())!=0:
        print("Layer Number:",i,"Layer Shape: ",lenet_model.layers[i].get_weights()[0].shape)

[<keras.layers.convolutional.Conv2D object at 0x7f3d52700eb8>, <keras.layers.pooling.AveragePooling2D object at 0x7f3d5269c080>, <keras.layers.convolutional.Conv2D object at 0x7f3d5269cf60>, <keras.layers.pooling.AveragePooling2D object at 0x7f3d526bd588>, <keras.layers.core.Flatten object at 0x7f3d52700160>, <keras.layers.core.Dense object at 0x7f3d51e550f0>, <keras.layers.core.Dense object at 0x7f3d51e693c8>, <keras.layers.core.Dense object at 0x7f3d51e0a5f8>]
Layer Number: 0 Layer Shape:  (5, 5, 1, 6)
Layer Number: 2 Layer Shape:  (5, 5, 6, 16)
Layer Number: 5 Layer Shape:  (400, 120)
Layer Number: 6 Layer Shape:  (120, 84)
Layer Number: 7 Layer Shape:  (84, 10)


### SVD Compression

In [0]:
# Loading Model
lenet_model.load_weights("Lenet.h5")
lenet_model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

# Loading weights of the model
keep_list=[0, 0, 14, 15, 10]
keep = keep_list[2]
fc1 = lenet_model.layers[5].get_weights()

# Decomposition and Reconstruction
U, S, V = np.linalg.svd(fc1[0], full_matrices=False)
tU, tS, tV = U[:, 0:keep], S[0:keep], V[0:keep, :]
fc1_t = np.matmul(np.matmul(tU, np.diag(tS)), tV)
print("Reconstruction Error: ",np.mean(abs(fc1[0]-fc1_t)))
print("Number of Elements: ",tU.size+tS.size+tV.size)

# Loading weights for new model
fc1[0] = fc1_t
lenet_model.layers[5].set_weights(fc1)

# Write the testing input and output variables
score = lenet_model.evaluate(x_test, y_test, verbose=0)
truncsvd_accuracy = score[1]
print('Test accuracy:', truncsvd_accuracy)

Reconstruction Error:  0.057772562
Number of Elements:  7294
Test accuracy: 0.9414


In [0]:
print(U.shape,'',S.shape,'',V.shape)
print(tU.shape,'',tS.shape,'',tV.shape)

(400, 120)  (120,)  (120, 120)
(400, 14)  (14,)  (14, 120)
